Ex 3):

In [1]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/test.nl
chmod u+rx ipopt

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  libgfortran5
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 589 kB of archives.
After this operation, 2,660 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libgfortran5 amd64 8.4.0-1ubuntu1~18.04 [589 kB]
Fetched 589 kB in 1s (509 kB/s)
Selecting previously unselected package libgfortran5:amd64.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../libgfortran5_8.4.0-1ubuntu1~18.04_amd64.deb ...
Unpacking libgfortran5:amd64 (8.4.0-1ubuntu1~18.04) ...
Setting up libgfortran5:amd64 (8.4.0-1ubuntu1~18.04) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [2]:
!./ipopt test.nl



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      230

Total number of variables............................:       30
                     variables with only lower bounds:       30
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

In [3]:
!pip install -q pyomo

     |████████████████████████████████| 9.1 MB 7.6 MB/s 
     |████████████████████████████████| 49 kB 6.6 MB/s 


In [4]:
import numpy as np

In [5]:
from pyomo.environ import *

In [6]:
import pandas as pd

In [7]:
data_csvfile = pd.read_csv('coef_lab06_ex3.csv')

In [8]:
model_ex3 = ConcreteModel()

In [9]:
N = 3
M = 6

In [10]:
col_indices = range(N)
col_indices_r = range(M)

In [11]:
model_ex3.x = Var(col_indices)
model_ex3.r = Var(col_indices_r)

In [12]:
model_ex3.obj = Objective(expr = (model_ex3.r[0])**2 + (model_ex3.r[1])**2 + (model_ex3.r[2])**2 + (model_ex3.r[3])**2 + (model_ex3.r[4])**2 + (model_ex3.r[5])**2 ,sense = minimize)


In [13]:
model_ex3.constraint = ConstraintList()

In [14]:
model_ex3.constraint.add(model_ex3.r[0]- (175000-summation(data_csvfile['P1'], model_ex3.x)) == 0)
model_ex3.constraint.add(model_ex3.r[1] - (150000-summation(data_csvfile['P2'], model_ex3.x)) == 0)
model_ex3.constraint.add(model_ex3.r[2] - (125000-summation(data_csvfile['P3'], model_ex3.x))== 0)
model_ex3.constraint.add(model_ex3.r[3] -  (90000-summation(data_csvfile['P4'], model_ex3.x))== 0)
model_ex3.constraint.add(model_ex3.r[4] -(80000-summation(data_csvfile['P5'], model_ex3.x))==0)
model_ex3.constraint.add(model_ex3.r[5] -(120000-summation(data_csvfile['P6'], model_ex3.x))==0)

In [15]:
model_ex3.pprint()

3 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    r_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

2 Var Declarations
    r : Size=6, Index=r_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals
          4 :  None :  None :  None : False :  True :  Reals
          5 :  None :  None :  None : False :  True :  Reals
    x : Size=3, Index=x_index
        Key

In [16]:
opt_ipopt = SolverFactory('ipopt')

In [22]:
result= opt_ipopt.solve(model_ex3)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model_ex3.obj())

print('\nConstraints:')
print(model_ex3.constraint.display())

print('\nDecision Variables')
for i in range(0,3):
  print('x[',i,'] = ', model_ex3.x[i].value)
for i in range(0,6):
  print('r[',i,'] = ', model_ex3.r[i].value)


Solver status: ok
Solver termination condition: optimal

Objective =  4291134028.807456

Constraints:
constraint : Size=6
    Key : Lower : Body                    : Upper
      1 :   0.0 : -2.9103830456733704e-11 :   0.0
      2 :   0.0 :  3.2741809263825417e-11 :   0.0
      3 :   0.0 :  -3.637978807091713e-11 :   0.0
      4 :   0.0 :   7.275957614183426e-12 :   0.0
      5 :   0.0 :  1.8189894035458565e-11 :   0.0
      6 :   0.0 : -1.0913936421275139e-11 :   0.0
None

Decision Variables
x[ 0 ] =  104904.41643719551
x[ 1 ] =  38.702075831391646
x[ 2 ] =  -193.32238932429567
r[ 0 ] =  48762.444397373416
r[ 1 ] =  14058.726964626174
r[ 2 ] =  -28366.606651133254
r[ 3 ] =  -26533.83816987926
r[ 4 ] =  -13331.391654310526
r[ 5 ] =  5410.6651133234445


**In the Regreesion problem:**

Objective =  4291134028.807456

**Decision Variables**:

x[ 0 ] =  104904.41643719551

x[ 1 ] =  38.702075831391646

x[ 2 ] =  -193.32238932429567

r[ 0 ] =  48762.444397373416

r[ 1 ] =  14058.726964626174

r[ 2 ] =  -28366.606651133254

r[ 3 ] =  -26533.83816987926

r[ 4 ] =  -13331.391654310526

r[ 5 ] =  5410.6651133234445


**In the absolute value problem:**

Objective (Optimal value of sum of residuals) =  125454.54499999993

**Decision Variables:**

x[ 0 ] =  104181.82  # x[ 0 ] represents b[ 0 ] which is as in the given problem.

x[ 1 ] =  67.272727  # x[ 1 ] represents b[ 1 ] which is as in the given problem.

x[ 2 ] =  -356.36364  # x[ 2 ] represents b[ 2 ] which is as in the given problem.

**Residuals:**

r[ 0 ] =  38818.182

r[ 1 ] =  -2.7009345e-11

r[ 2 ] =  54272.727

r[ 3 ] =  32363.636

r[ 4 ] =  -2.7785522e-11

r[ 5 ] =  -1.671118e-11



In [23]:
residual_list_abs = [ '38818.182','-2.7009345e-11', '54272.727','32363.636', '-2.7785522e-11', '-1.671118e-11']
residual_list_regression =['8762.444397373416','14058.726964626174','-28366.606651133254','-26533.83816987926','-13331.391654310526','5410.6651133234445']
a = max(residual_list_abs)
b = max(residual_list_regression) 
print(a,b)

54272.727 8762.444397373416


Maximum value of the residual in absolute value method is less than regression method, which is obvious , because there we used only absolute value and here we are suqaring that value, so squared will be more than absolute value.

All the values in the objective functions are sum of squared values of residuals ,which will lead to the larger optimal value as compared to the one in which we used sum of absolute values of residuals.

Basically our motive is to find  the least error. If the errors are squared, it will be easy to differentiate between the errors comparing to taking the absolute value of the error.Also linear regression gives best fit line as compared to other one.